# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [1]:
#Importando o excel 
import pandas as pd

lacta=pd.read_excel('tweets_lacta.xlsx')

#Separando os tweets em tweets relevantes e irrelevantes
relevant_tweets=lacta[lacta['Classificador - 1 para relevante(críticas/sugestões) para a empresa e 0 para não relevante']==1]

irrelevant_tweets=lacta[lacta['Classificador - 1 para relevante(críticas/sugestões) para a empresa e 0 para não relevante']==0]

In [2]:
#Exceções - lista que contém elementos que devem ser retirados dos tweets para melhor análise 
excecoes=[':',';','[',']', '{','}','—','-','+','=','/','|', ')','_', '(','@','#', '?','.', '!', ' ',',','“','”','\\\\']
#Lista das palavras separadas de todos os tweets relevantes
lista_palavras_relevant=[]

for elemento in relevant_tweets['Treinamento']:
    quebra=elemento.split()
    for palavra in quebra:
        lista_letra=list(palavra)
        letras_corretas=[]
        for letra in lista_letra:
            if letra not in excecoes:
                letras_corretas.append(letra)
        palavra_limpa=''.join(letras_corretas)
        lista_palavras_relevant.append(palavra_limpa)
        

#Lista das palavras separadas de todos os tweets irrelevantes
lista_palavras_irrelevant=[]

for elemento in irrelevant_tweets['Treinamento']:
    quebra=elemento.split()
    for word in quebra:
        letter_list=list(word)
        correct_words=[]
        for letter in letter_list:
            if letter not in excecoes:
                correct_words.append(letter)
        clean_word=''.join(correct_words)
        lista_palavras_irrelevant.append(clean_word)

        
#Após a separação feita anteriormente, algumas strings ficaram vazias. Logo, o programa a seguir retira as palavras vazias.
for palavra in lista_palavras_relevant:
    if palavra=='':
        lista_palavras_relevant.remove(palavra)

for palavra in lista_palavras_irrelevant:
    if palavra=='':
        lista_palavras_irrelevant.remove(palavra)

In [3]:
#Fazendo a contagem de cada palavra em relevantes e irrelevantes

palavras_relevantes={}

palavras_irrelevantes={}


for palavra in lista_palavras_relevant:
    if palavra not in palavras_relevantes:
        palavras_relevantes[palavra]=2
    else:
        palavras_relevantes[palavra]+=1

for palavra in lista_palavras_irrelevant:
    if palavra not in palavras_irrelevantes:
        palavras_irrelevantes[palavra]=2
    else:
        palavras_irrelevantes[palavra]+=1
        
#Total de palavras no excel de tweets relevantes
total_relev=sum(palavras_relevantes.values())

#Total de palavras no excel de tweets irrelevantes
total_irrelev=sum(palavras_irrelevantes.values())

#União das duas, sem repetição
lista_comum=list(palavras_relevantes.keys())+list(palavras_irrelevantes.keys())
lista_sem_rep=set(lista_comum)
length_lista=len(lista_sem_rep)

In [4]:
#Probabilidade de um tweet ser relevante ou não
p1=len(relevant_tweets)/len(lacta['Treinamento'])
p0=len(irrelevant_tweets)/len(lacta['Treinamento'])

#Encontrando a probabilidade de cada palavra
for palavra in palavras_relevantes:
    palavras_relevantes[palavra]=palavras_relevantes[palavra]/total_relev
    
for palavra in palavras_irrelevantes:
    palavras_irrelevantes[palavra]=palavras_irrelevantes[palavra]/total_irrelev
palavras_irrelevantes

{'mano': 0.0011605415860735009,
 'esse': 0.0019342359767891683,
 'chocolate': 0.017988394584139266,
 'é': 0.01276595744680851,
 'maravilhoso': 0.0007736943907156673,
 'amem': 0.00038684719535783365,
 'lacta': 0.04700193423597679,
 'rt': 0.009284332688588007,
 'oalezoun': 0.005415860735009671,
 'aquela': 0.006963249516441006,
 'marca': 0.006189555125725338,
 'de': 0.03210831721470019,
 'q': 0.00793036750483559,
 'aponta': 0.006189555125725338,
 'onde': 0.0065764023210831725,
 'tá': 0.006963249516441006,
 'as': 0.007156673114119922,
 'coisas': 0.0065764023210831725,
 'quem': 0.0013539651837524177,
 'prefere': 0.00038684719535783365,
 'qualquer': 0.0019342359767891683,
 'caixa': 0.005802707930367505,
 'bombom': 0.0027079303675048355,
 'em': 0.003481624758220503,
 'vez': 0.0013539651837524177,
 'louquissimo': 0.00038684719535783365,
 'gabirigobeli': 0.00038684719535783365,
 'vamo': 0.00038684719535783365,
 'tomar': 0.0009671179883945841,
 'um': 0.005996131528046422,
 'aqui': 0.001160541586

In [5]:
#Criando a função para o classificador

def classificador(excel):
    classificacao=[]
    for tweet in excel['Teste']:
        quebra_tudo=[]
        quebra=tweet.split()
        p_r=p1
        p_i=p0
        for palavra in quebra:
            lista_letra=list(palavra)
            letras_corretas=[]
            for letra in lista_letra:
                if letra not in excecoes:
                    letras_corretas.append(letra)
            palavra_limpa=''.join(letras_corretas)
            quebra_tudo.append(palavra_limpa)
        for item in quebra_tudo:
            if item not in palavras_relevantes.keys() and item in palavras_irrelevantes.keys():
                p_r*=1/(total_relev+length_lista)
                p_i*=palavras_irrelevantes[item]
            elif item in palavras_relevantes.keys() and item not in palavras_irrelevantes.keys():
                p_i*=1/(total_irrelev+length_lista)
                p_r*=palavras_relevantes[item]
            elif item in palavras_relevantes.keys() and item in palavras_irrelevantes.keys():
                p_r*=palavras_relevantes[item]
                p_i*=palavras_irrelevantes[item]
            else:
                p_r*=1/(total_relev+length_lista)
                p_i*=1/(total_irrelev+length_lista)
        if p_r>p_i:
            classi=1
        else:
            classi=0
        classificacao.append(classi)
    excel['Naive']=classificacao
    excel.to_excel('Up.xlsx')

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [6]:
teste = pd.read_excel('tweets_lacta.xlsx',sheet_name="Teste")
classificador(teste)

In [16]:
a=pd.read_excel('Up.xlsx')
a
negativos=0
positivos=0
falsos_positivos=0
falsos_negativos=0
for n in range(len(a['Teste'])):
    if a['Classificador - 1 para relevante(críticas/sugestões) para a empresa e 0 para não relevante'][n]==1 and a['Naive'][n]==1:
        positivos+=1
    elif a['Classificador - 1 para relevante(críticas/sugestões) para a empresa e 0 para não relevante'][n]==1 and a['Naive'][n]==0:
        falsos_negativos+=1
    elif a['Classificador - 1 para relevante(críticas/sugestões) para a empresa e 0 para não relevante'][n]==0 and a['Naive'][n]==1:
        falsos_positivos+=1
    else:
        negativos+=1
print("Positivos: {0}, {1}%".format(positivos,(positivos*100/len(a['Teste']))))
print("Negativos: {0}, {1}%".format(negativos,(negativos*100/len(a['Teste']))))
print("Falsos-negativos: {0}, {1}%".format(falsos_negativos,(falsos_negativos*100/len(a['Teste']))))
print("Falsos-positivos: {0}, {1}%".format(falsos_positivos,(falsos_positivos*100/len(a['Teste']))))

Positivos: 28, 0.14%.2f
Negativos: 108, 0.54%.2f
Falsos-negativos: 2, 1.00%
Falsos-positivos: 62, 31.00%


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
